# Wikipedia Corpus Based Language Models

In [2]:
wiki_corpus_path = "C:\\Users\\LpySHiN\\AI-NLP\\wiki_divide\\AA"

In [3]:
import os
os.path.exists(wiki_corpus_path)

True

In [4]:
import pandas as pd

In [9]:
# 编码检测
import chardet, csv
from chardet.universaldetector import UniversalDetector

# with open(database, 'rb') as k:
#     print(chardet.detect(k.read(10000)))

usock = open(wiki_corpus_path, 'rb')
detector = UniversalDetector()
for line in usock.readlines():
    detector.feed(line)
    if detector.done:
        break
detector.close()
usock.close()
print(detector.result)
# chardet_rst = pd.read_csv(database, encoding='GB2312')

{'encoding': 'Windows-1254', 'confidence': 0.24576554947717053, 'language': 'Turkish'}


In [5]:
import glob
wiki_corpus_path_pattern = wiki_corpus_path + '\\wiki_**'
files = glob.glob(wiki_corpus_path_pattern)

In [6]:
# 读取wiki->list
all_articles = list()
for name in files:
    try:
        f = open(name, 'r', encoding='utf8')
        lines = f.readlines()
        all_articles.append(lines)
    except Exception as E:
        print(E.message)

In [7]:
import re

In [12]:
def find_all_tokens(string):
    return ' '.join(re.findall('[\w|\d]+', string))

In [9]:
sum_len = 0
for articles in all_articles:
    sum_len += len(articles)
    print(len(articles))
sum_len
print(0,sum_len)

696332
836072
984568
1037490
1158947
1173605
1175800
1908933
1501687
1192449
1237096
1224260
763134
0 14890373


In [10]:
# 预处理
TEXT = ''
i = 0
for articles in all_articles[:3]:
    print(i, len(articles))
    file_count = 0
    for element in articles:
        if (file_count % 100000) == 0:
            print('File[{}] Finish {}%'.format(i + 1, file_count * 100.00 / len(articles)))
        file_count += 1
        if re.match('<doc id|</doc>', element) is not None:
            continue
        else:
            TEXT += element
    if file_count == len(articles):
        print('Finish [{}]'.format(i))
    i += 1
        

0 696332
File[1] Finish 0.0%
File[1] Finish 14.360965746224503%
File[1] Finish 28.721931492449006%
File[1] Finish 43.0828972386735%
File[1] Finish 57.44386298489801%
File[1] Finish 71.80482873112251%
File[1] Finish 86.165794477347%
Finish [0]
1 836072
File[2] Finish 0.0%
File[2] Finish 11.960692380560525%
File[2] Finish 23.92138476112105%
File[2] Finish 35.88207714168158%
File[2] Finish 47.8427695222421%
File[2] Finish 59.80346190280263%
File[2] Finish 71.76415428336315%
File[2] Finish 83.72484666392369%
File[2] Finish 95.6855390444842%
Finish [1]
2 984568
File[3] Finish 0.0%
File[3] Finish 10.156738793054416%
File[3] Finish 20.31347758610883%
File[3] Finish 30.470216379163247%
File[3] Finish 40.62695517221766%
File[3] Finish 50.78369396527208%
File[3] Finish 60.940432758326494%
File[3] Finish 71.09717155138091%
File[3] Finish 81.25391034443533%
File[3] Finish 91.41064913748974%
Finish [2]


In [14]:
TEXT_TOKENS = find_all_tokens(TEXT)

In [15]:
# 繁体->简体
from hanziconv import HanziConv
TEXT_TOKENS = HanziConv.toSimplified(TEXT_TOKENS)

In [17]:
#
import jieba
def cut(string):
    return list(jieba.cut(string))

ALL_TOKENS = [i for i in cut(TEXT_TOKENS) if i.strip()]

## Get the frequeces of words

In [20]:
from collections import Counter

In [22]:
words_count = Counter(ALL_TOKENS)

In [23]:
words_count.most_common(10)

[('的', 2809970),
 ('在', 764020),
 ('年', 674711),
 ('是', 540321),
 ('和', 420283),
 ('了', 363728),
 ('为', 350624),
 ('月', 282922),
 ('与', 270505),
 ('有', 248615)]

In [25]:
frequences_all = [f for w, f in words_count.most_common()]

In [26]:
frequences_sum = sum(frequences_all)

In [28]:
ESP = 1 / frequences_sum

## 1-gram

In [42]:
def get_prob(word):
    if word in words_count:
        return words_count[word] / frequences_sum
    else:
        return ESP

In [39]:
from functools import reduce

In [40]:
def product(numbers):
    return reduce(lambda n1, n2: n1 * n2, numbers)

In [41]:
product([1,2,3])

6

In [43]:
def lm_1_gram(string):
    words = cut(string)
    return product(get_prob(w) for w in words)

In [44]:
lm_1_gram('我真是太累了')

1.3929063423019209e-18

In [45]:
need_compared = [
    "今天晚上请你吃大餐，我们一起吃日料 明天晚上请你吃大餐，我们一起吃苹果",
    "真事一只好看的小猫 真是一只好看的小猫",
    "我去吃火锅，今晚 今晚我去吃火锅"
]

for s in need_compared:
    s1, s2 = s.split()
    p1, p2 = lm_1_gram(s1), lm_1_gram(s2)
    
    better = s1 if p1 > p2 else s2
    
    print('{} is more possible'.format(better))
    print('-'*4 + ' {} with probility {}'.format(s1, p1))
    print('-'*4 + ' {} with probility {}'.format(s2, p2))

明天晚上请你吃大餐，我们一起吃苹果 is more possible
---- 今天晚上请你吃大餐，我们一起吃日料 with probility 4.086292655830849e-54
---- 明天晚上请你吃大餐，我们一起吃苹果 with probility 6.032069023718534e-52
真是一只好看的小猫 is more possible
---- 真事一只好看的小猫 with probility 5.328096244532081e-26
---- 真是一只好看的小猫 with probility 3.720357960077989e-23
今晚我去吃火锅 is more possible
---- 我去吃火锅，今晚 with probility 1.2894577206106787e-28
---- 今晚我去吃火锅 with probility 6.583922766773603e-21


## 2-gram

In [55]:
all_2_grams_words = [''.join(ALL_TOKENS[i:i + 2]) for i in range(len(ALL_TOKENS[:-2]))]

In [47]:
_2_gram_sum = len(all_2_grams_words)
_2_gram_counter = Counter(all_2_grams_words)

def get_combination_prob(w1, w2):
    if w1 + w2 in _2_gram_counter: return _2_gram_counter[w1+w2] / _2_gram_sum
    else:
        return 1 / _2_gram_sum

In [48]:
def get_prob_2_gram(w1, w2):
    return get_combination_prob(w1, w2) / get_prob(w1)

In [50]:
get_prob_2_gram('写', '书包')

0.0001248439499587882

In [51]:
def lm_2_gram(sentence):
    sentence_prob = 1
    words = cut(sentence)
    
    for i, word in enumerate(words):
        if i == 0:
            prob = get_prob(word)
        else:
            previous = words[i - 1]
            prob = get_prob_2_gram(previous, word)
        sentence_prob *= prob
    return sentence_prob

In [52]:
need_compared = [
    "今天晚上请你吃大餐，我们一起吃日料 明天晚上请你吃大餐，我们一起吃苹果",
    "真事一只好看的小猫 真是一只好看的小猫",
    "今晚我去吃火锅 今晚火锅去吃我",
    "洋葱奶昔来一杯 养乐多绿来一杯",
    "今天天气居然是晴天 大使馆天气居然是乌鸦"
]

for s in need_compared:
    s1, s2 = s.split()
    p1, p2 = lm_2_gram(s1), lm_2_gram(s2)
    
    better = s1 if p1 > p2 else s2
    
    print('{} is more possible'.format(better))
    print('-'*4 + ' {} with probility {}'.format(s1, p1))
    print('-'*4 + ' {} with probility {}'.format(s2, p2))

今天晚上请你吃大餐，我们一起吃日料 is more possible
---- 今天晚上请你吃大餐，我们一起吃日料 with probility 5.182488934711886e-29
---- 明天晚上请你吃大餐，我们一起吃苹果 with probility 7.403555621016978e-30
真是一只好看的小猫 is more possible
---- 真事一只好看的小猫 with probility 4.4584441739465745e-21
---- 真是一只好看的小猫 with probility 1.5825246975550127e-17
今晚我去吃火锅 is more possible
---- 今晚我去吃火锅 with probility 1.7768601708927513e-14
---- 今晚火锅去吃我 with probility 2.865662810946662e-16
养乐多绿来一杯 is more possible
---- 洋葱奶昔来一杯 with probility 3.1714803518278e-13
---- 养乐多绿来一杯 with probility 1.9584947565513632e-08
今天天气居然是晴天 is more possible
---- 今天天气居然是晴天 with probility 2.0854367062102974e-14
---- 大使馆天气居然是乌鸦 with probility 8.082461777756108e-18


## 3-gram
内存不够跑了，老师帮忙看下有没有问题吧~

In [61]:
all_3_grams_words = [''.join(ALL_TOKENS[i:i + 3]) for i in range(len(ALL_TOKENS[:-3]))]

MemoryError: 

In [ ]:
_3_gram_sum = len(all_3_grams_words)
_3_gram_counter = Counter(all_3_grams_words)

def get_3_combination_prob(w1, w2, w3):
    if w1 + w2 + w3 in _3_gram_counter: return _3_gram_counter[w1 + w2 + w3] / _3_gram_sum
    else:
        return 1 / _3_gram_sum

In [ ]:
def get_prob_3_gram(w1, w2, w3):
    return get_3_combination_prob(w1, w2, w3) / get_combination_prob(w1, w2)

In [ ]:
def lm_3_gram(sentence):
    sentence_prob = 1
    words = cut(sentence)
    
    for i, word in enumerate(words):
        if i == 0:
            prob = get_prob(word)
        elif i == 1:
            prob = get_prob_2_gram(words[i - 1], word)
        else:
            prob = get_prob_2_gram(words[i - 2], words[i - 3], word)
        sentence_prob *= prob
    return sentence_prob

## Step 4

PASS

## Step 5

### If we need to solve following problems, how can language model help us?

+ Voice Recognization

语言模型可将语音识别转化的文本进行纠错，通过分析低概率词组进行更正

+ Sogou pinyin input

优化语言模型，新增pinyin-word字典，通过语言模型计算pinyin生成句的概率，按概率排序推荐为输入选项

+ Auto correction in search engine

判断用户搜索输入的句子，通过语言模型对句子中概率较低的字/两连词组/三连词组进行替换，再进行搜索并推荐搜索纠错后的句子

+ Abnormal Detection

例如在语音对话机器人模型中，针对语音输入部分进行语言模型分析，将概率过低的输入作为异常分支，采用特殊回复处理此分支。

### Compared to the previous learned parsing and pattern match problems. What's the advantage and disavantage of Probability Based Methods?

Ans:

优点： 基于概率的方法编码较为容易，可以解决大部分问题，而且准确率很高；

缺点： 基于概率的方法其本质是基于语料库，语料库的概率特性将会影响概率模型的分析结果。

## How to solve *OOV* problem?

### Q1: How did you solve this problem in your programming task?

Ans:

+ 通过返回固定概率(1/frequece_sum)表征OOV词组/字

Others method:
    
+ 增大语料库丰富性
+ 在语言模型输入时自动检测OOV词组，将该词组用同类别词语替代后再进行语言模型分析

### Q2: Read about the 'Good-Turing Estimator', can explain the main points about this method, and may implement this method in your programming task

老师如果能看到这里的话，我英语比较差，没看到Good-Turing的那个method，没明白每个公式的参数的含义，如果可以的话，能不能出个翻译版或者课上单独讲一下呀？如下，每个参数的含义。

$$ N_{r}\;=\;\vert \{ x\;\vert\;R_{x}=\;r\} \vert $$

$$ N\;=\;\sum_{r=1}^{\infty}{r \cdot Nr}$$

$$ p_0\;=\;\frac{N1}{N}$$

$$ p_r\;=\;\frac{(r+1) \cdot S(N_{r+1})}{N \cdot S(N_r)}$$

$$ z_r\;=\;\frac{N_r}{0.5 \cdot (t\,-\,q)}$$